In [1]:
import pandas as pd
import numpy as np
import torch
import h5py
import sys
import cvxpy as cp


In [2]:
# 一些参数，先进行测试，后面可能会由我每天传入
class params:
    trade_time_list = ["1129", "1429"]
    today_open_trade_task = pd.DataFrame()
    pt_asset_1129 = 19994192.0
    pt_asset_1429 = 20075239.14
    citic_share_limit = 0.05
    cmvg_share_limit = 0.075
    pt_tvr_limit = 15.0
    rq_tvr_limit = 40.0
    pt_max_prop_total = 2.0
    rq_max_prop_total = 5.0

def noteq(a, b, eps=1e-8):# 不等于
    return abs(a-b) >= eps

def my_round2(x):  # 此处是因为python浮点数不精确
    return round(x + 1e-10, 2)

# 支持文件，由我每天传入
open_support_data = pd.read_csv(rf"./Open_Support_20250221.csv").set_index("Code")
open_support_data.index = open_support_data.index.map(lambda x:str(x).zfill(6))

# 可以交易的股票池
params.today_istrade_stock_list = list(open_support_data.loc[open_support_data.isTrade == 1].index)


In [3]:
time = "1129"
stock_hold_all = pd.read_csv(rf"./stock_hold_20250221_{int(time) * 100000}.csv").set_index("Code")
stock_hold_all.index = stock_hold_all.index.map(lambda x:str(x).zfill(6))
stock_hold_pt = stock_hold_all["stock_hold_pt"].copy() # 当前多头账户持仓
stock_hold_rq = stock_hold_all["stock_hold_rq"].copy() # 当前空头账户持仓
stock_available_pt = stock_hold_all["stock_available_pt"].copy() # 当前多头可以卖出的持仓
stock_available_rq = stock_hold_all["stock_available_rq"].copy() # 当前空头可以还券的持仓
stock_rq = stock_hold_all["stock_rq"].copy()  # 当前券池实时可融余额

# 共同股票池子,为持仓股票+可融股票+可与交易股票并集
stock_list = sorted(list(set(params.today_istrade_stock_list) | set(stock_hold_pt.index) | set(stock_hold_rq.index)| set(stock_rq.index)))


# 市值分组、行业分组生成的哑变量矩阵，后面组合优化会用
cmvg_mtx = pd.get_dummies(open_support_data['Barra_nmv_g'].reindex(stock_list).fillna(7)).T.sort_index(axis=0).astype(int).fillna(0)
cmvg_mtx = cmvg_mtx.values
citic_mtx = pd.get_dummies(open_support_data['Barra_citic'].reindex(stock_list).fillna(7)).T.sort_index(axis=0).astype(int).fillna(0)
citic_mtx = citic_mtx.values


# 获取当前模型打分
model_score = stock_hold_all["model_score"].copy()
ori_score_array = model_score.reindex(stock_list).rank(pct = True).fillna(0.5)

# 获取此时股票价格、股票流动性, get_can_trade_amount这个函数稍后给出，先直接传入
# stock_price_now, stock_buy_liquid, stock_sell_liquid, score_array = get_can_trade_amount(stock_list, time, ori_score_array , params)

stock_buy_liquid  = stock_hold_all["stock_buy_liquid"].copy()
stock_sell_liquid  = stock_hold_all["stock_sell_liquid"].copy()
stock_price_now  = stock_hold_all["stock_price_now"].copy()
score_array = stock_hold_all["score_array"].copy()

stock_buy_liquid = stock_buy_liquid.reindex(stock_list).fillna(0)
stock_sell_liquid = stock_sell_liquid.reindex(stock_list).fillna(0)
stock_price_now = stock_price_now.reindex(stock_list).fillna(0)
stock_hold_pt = pd.Series(stock_hold_pt).reindex(stock_list).fillna(0) # 当前持仓
stock_hold_rq = pd.Series(stock_hold_rq).reindex(stock_list).fillna(0) # 当前持仓
stock_rq = stock_rq.reindex(stock_list).fillna(0) # 当前可融余额

In [4]:
def my_int(x, int_num):
    return x // int_num * int_num

In [5]:
params.today_open_trade_task = pd.DataFrame()
stock_order = pd.read_csv(rf"./stock_order_1129.csv")
stock_order = stock_order.set_index("Unnamed: 0")['0']
stock_order.index = stock_order.index.map(lambda x:str(x).zfill(6))

# 这一时点的交易任务，通过算法实现
today_open_trade_task = []
# 获取当前可以多头买入和融券卖出的额度, 直接通过api获取
today_can_ptbuy_amt = 10000
today_can_rqsell_amt = 10000
stock_order_left = stock_order.copy()

# 先进行空头还券、多头卖出
for stock, a_estimate in stock_order.iteritems():
    ## 风控和取整设定
    int_num = 1 if stock[:2] == '68' else 100 # 取整单位
    min_num = 200 if stock[:2] == '68' else 100 # 最小股数

    if a_estimate < 0: # 卖出阶段，有股票先卖股票，没股票再融券
        p_order = stock_price_now[stock]
        v_estimate = - a_estimate / p_order # 盘口挂单量

        # 先查询股票是否存在
        v_avai_pt = stock_available_pt[stock]  # 查询普通户持仓
        if v_avai_pt > 0:
            open_trade_task = {}
            # 截止到此时，p_order和v_order都已给出，该股票的交易任务已经确定
            v_order = my_int(min(v_avai_pt, v_estimate),  int_num)
            if v_order > min_num: # 如果小于最小值没法卖
                a_order = p_order * v_order
                open_trade_task['code'] = stock
                open_trade_task['type'] = "PT"
                open_trade_task['side'] = "S"
                open_trade_task['market'] = "SZ" if stock[0] in ['0', '3'] else "SH"
                open_trade_task['v_plan'] = v_order
                open_trade_task['p0'] = p_order
                today_open_trade_task.append(open_trade_task) 
                stock_order_left[stock] += v_order * p_order
                today_can_ptbuy_amt += v_order * p_order * 0.997

    elif a_estimate > 0: # 买入阶段
        p_order = stock_price_now[stock]
        v_estimate = a_estimate / p_order # 盘口挂单量

        v_avai_rq = - stock_available_rq[stock]  # 查询融券户持仓
        if v_avai_rq > 0:
            open_trade_task = {}
            # 截止到此时，p_order和v_order都已给出，该股票的交易任务已经确定
            v_order = my_int(min(v_avai_rq, v_estimate),  int_num)
            if v_order > min_num: # 如果小于最小值没法卖
                a_order = p_order * v_order
                open_trade_task['code'] = stock
                open_trade_task['type'] = "RQ"
                open_trade_task['side'] = "B"
                open_trade_task['market'] = "SZ" if stock[0] in ['0', '3'] else "SH"
                open_trade_task['v_plan'] = v_order
                open_trade_task['p0'] = p_order
                today_open_trade_task.append(open_trade_task) 
                stock_order_left[stock] -= v_order * p_order
                today_can_rqsell_amt += v_order * p_order * 0.997


# 对剩余的进行多头买入、空头卖出
for stock, a_estimate in stock_order_left.iteritems():
    ## 风控和取整设定
    int_num = 1 if stock[:2] == '68' else 100 # 取整单位
    min_num = 200 if stock[:2] == '68' else 100 # 最小股数

    if a_estimate < 0: # 卖出阶段，有股票先卖股票，没股票再融券
        p_order = stock_price_now[stock]
        v_estimate = - a_estimate / p_order # 盘口挂单量

        if stock_rq[stock] > 0: # 如果存在可融余额，这一步需要锁券再卖出
            open_trade_task = {}
            # 截止到此时，p_order和v_order都已给出，该股票的交易任务已经确定
            v_order = my_int(min(stock_rq[stock], v_estimate, today_can_rqsell_amt / p_order),  int_num)
            if v_order > min_num: # 如果小于最小值没法卖
                a_order = p_order * v_order
                open_trade_task['code'] = stock
                open_trade_task['type'] = "RQ"
                open_trade_task['side'] = "S"
                open_trade_task['market'] = "SZ" if stock[0] in ['0', '3'] else "SH"
                open_trade_task['v_plan'] = v_order
                open_trade_task['p0'] = p_order
                today_open_trade_task.append(open_trade_task)     
                today_can_rqsell_amt -= v_order * p_order * 0.997

    elif a_estimate > 0: # 买入阶段
        p_order = stock_price_now[stock]
        v_estimate = a_estimate / p_order # 盘口挂单量
        # 还券之后继续买入
        open_trade_task = {}
        # 截止到此时，p_order和v_order都已给出，该股票的交易任务已经确定
        v_order = my_int(min(v_estimate, today_can_ptbuy_amt / p_order) ,   int_num)
        if v_order > min_num: # 如果小于最小值没法卖
            a_order = p_order * v_order
            open_trade_task['code'] = stock
            open_trade_task['type'] = "PT"
            open_trade_task['side'] = "B"
            open_trade_task['market'] = "SZ" if stock[0] in ['0', '3'] else "SH"
            open_trade_task['v_plan'] = v_order
            open_trade_task['p0'] = p_order
            today_open_trade_task.append(open_trade_task)  
            today_can_ptbuy_amt -= v_order * p_order * 1.003

params.today_open_trade_task = pd.concat([params.today_open_trade_task, pd.DataFrame(today_open_trade_task)])
params.today_open_trade_task.reset_index(drop = True).to_csv(rf"stock_order_trade_1129.csv")